<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#PDC-2020-and-2021" data-toc-modified-id="PDC-2020-and-2021-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>PDC 2020 and 2021</a></span></li><li><span><a href="#BINET-Logs" data-toc-modified-id="BINET-Logs-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>BINET Logs</a></span></li></ul></div>

# Dataset Descriptions:

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

## PDC 2020 and 2021

In [ ]:
#export
from dapnn.imports import *
from dapnn.data_processing import *

from dapnn.anomaly import *

/home/lahann/miniconda3/envs/dapnn/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370172916/work/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [ ]:
def get_ds_stats(path):
    res=[]
    for log in progress_bar(glob.glob(path)):
        splits = log.split('.')[0].split('_')
        log_name = splits[-1]
        log = import_log(log)
        number_cases = len(log.index.unique())
        number_activities= len(log.activity.unique())
        number_events = len(log)
        number_attr = 1
        number_anomalies= len(log[log['normal']==False]['trace_id'].unique()) if 'normal' in log.columns else 'unknown'
        res.append([log_name,number_cases,number_activities,number_events
                    ,number_anomalies])
    res= pd.DataFrame(res,columns=['Name','# Cases','# Activities','# Events','# Anomalies'])
    return res

In [ ]:
pdc20_train= get_ds_stats('data/csv/PDC2020_training/*')
pdc20_train

,Name,# Cases,# Activities,# Events,# Anomalies
0,1001001,1000,18,14013,unknown
1,0111100,1000,26,20873,unknown
2,0000011,1000,20,17986,unknown
3,1000010,1000,20,18000,unknown
4,0201010,1000,24,49183,unknown
...,...,...,...,...,...
187,1200101,1000,38,54942,unknown
188,0111001,1000,26,22930,unknown
189,0110111,1000,31,26168,unknown
190,1210101,1000,38,36955,unknown


In [ ]:
pdc20_gt= get_ds_stats('data/csv/PDC2020_ground_truth/*')
pdc20_gt

,Name,# Cases,# Activities,# Events,# Anomalies
0,1001001,1000,18,13997,490
1,0111100,1000,26,20169,431
2,0000011,1000,20,18017,504
3,1000010,1000,20,18044,500
4,0201010,1000,24,49549,490
...,...,...,...,...,...
187,1200101,1000,38,52593,500
188,0111001,1000,26,22071,470
189,0110111,1000,31,26471,437
190,1210101,1000,38,38648,486


In [ ]:
def format_ds(x):
    min = x.min()   
    max = x.max()
    return  min if min == max else f'{x.min()}-{x.max()}'

pdc20_train_stats = pdc20_train.apply(format_ds).to_frame().T
pdc20_gt_stats = pdc20_gt.apply(format_ds).to_frame().T


In [ ]:
pdc21_train= get_ds_stats('data/csv/PDC2021_training/*')
pdc21_train

,Name,# Cases,# Activities,# Events,# Anomalies
0,0010011,1000,45,14546,unknown
1,0200003,1000,65,28379,unknown
2,1001114,1000,37,10386,unknown
3,1110100,1000,62,22452,unknown
4,0100113,1000,53,22650,unknown
...,...,...,...,...,...
475,1001014,1000,37,12385,unknown
476,1100110,1000,53,23701,unknown
477,1000004,1000,52,14792,unknown
478,0010101,1000,52,12433,unknown


In [ ]:
pdc21_gt= get_ds_stats('data/csv/PDC2021_ground_truth/*')
pdc21_gt

,Name,# Cases,# Activities,# Events,# Anomalies
0,100011,250,42,7971,125
1,120100,250,49,10340,125
2,121110,250,45,10280,125
3,111010,250,60,10756,125
4,121010,250,59,11860,125
...,...,...,...,...,...
91,001100,250,42,7347,125
92,000011,250,44,8289,125
93,020011,250,53,10595,125
94,110100,250,47,9375,125


In [ ]:
pdc21_train_stats = pdc21_train.apply(format_ds).to_frame().T
pdc21_gt_stats = pdc21_gt.apply(format_ds).to_frame().T


In [ ]:
res =pd.concat([pdc20_train_stats,pdc20_gt_stats,pdc21_train_stats,pdc21_gt_stats])
res =res[res.columns[1:]]
res.index = ['PDC 2020 Train', 'PDC 2020 Test','PDC 2021 Train','PDC 2021 Test']
res['# Logs'] = [len(pdc20_train),len(pdc20_gt),len(pdc21_train),len(pdc21_gt)]
res =res[['# Logs','# Cases','# Activities','# Events','# Anomalies']]
res


,# Logs,# Cases,# Activities,# Events,# Anomalies
PDC 2020 Train,192,1000,16-38,8867-70106,unknown
PDC 2020 Test,192,1000,16-38,8764-68706,412-515
PDC 2021 Train,480,1000,37-65,9867-32009,unknown
PDC 2021 Test,96,250,35-64,6612-11860,125


In [ ]:
print(res.to_latex())

\begin{tabular}{lrllll}
\toprule
{} &  \# Logs & \# Cases & \# Activities &    \# Events & \# Anomalies \\
\midrule
PDC 2020 Train &     192 &    1000 &        16-38 &  8867-70106 &     unknown \\
PDC 2020 Test  &     192 &    1000 &        16-38 &  8764-68706 &     412-515 \\
PDC 2021 Train &     480 &    1000 &        37-65 &  9867-32009 &     unknown \\
PDC 2021 Test  &      96 &     250 &        35-64 &  6612-11860 &         125 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_4188149/138774964.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(res.to_latex())


## BINET Logs

In [ ]:
res=[]
for fn in progress_bar(glob.glob('data/csv/binet_logs/*')):
    log_name = fn.split('/')[-1][:-7]
    log = import_log(fn)
    number_cases = len(log.index.unique())
    number_activities= len(log.activity.unique())
    number_events = len(log)
    number_attr =len(get_attr(attr_dict,fn))-1
    number_anomalies= len(log[log['anomaly']!='normal']['trace_id'].unique())
    res.append([log_name,number_cases,number_activities,number_events,number_attr,number_anomalies])
binet_logs= pd.DataFrame(res,columns=['Name','# Cases','# Activities','# Events','# Attributes','# Anomalies'])
binet_logs

/home/lahann/dapnn/dapnn/data_processing.py:49: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(log_path)
/home/lahann/dapnn/dapnn/data_processing.py:49: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(log_path)
/home/lahann/dapnn/dapnn/data_processing.py:49: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(log_path)
/home/lahann/dapnn/dapnn/data_processing.py:49: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(log_path)
/home/lahann/dapnn/dapnn/data_processing.py:49: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(log_path)


,Name,# Cases,# Activities,# Events,# Attributes,# Anomalies
0,bpic15-0.3-4,1053,417,49542,6,307
1,large-0.3-4,5000,85,63385,4,1491
2,small-0.3-4,5000,41,56695,4,1529
3,bpic13-0.3-3,819,11,4068,7,162
4,medium-0.3-3,5000,65,38990,3,1534
5,medium-0.3-1,5000,65,41991,1,1488
6,bpic17-0.3-1,31509,53,1269176,2,9398
7,wide-0.3-3,5000,69,39965,3,1513
8,large-0.3-2,5000,85,61789,2,1482
9,bpic17-0.3-2,42995,17,285211,2,13193


In [ ]:
groups = np.unique(binet_logs['Name'].apply(lambda x: x.split('-')[0]).values)

In [ ]:
dfs =[]
for group in groups:
    df = binet_logs[binet_logs.Name.str.contains(group)]
    len_df=len(df)
    df = df.apply(format_ds).to_frame().T
    df['# Logs'] = [len_df]
    dfs.append(df)
res =pd.concat(dfs)
res =res[res.columns[1:]]
res.index=[i.upper() if 'bpi' in i else i.capitalize() for i in groups ]
res = res[['# Logs','# Cases','# Activities','# Events','# Attributes','# Anomalies']].sort_index()
res

,# Logs,# Cases,# Activities,# Events,# Attributes,# Anomalies
BPIC12,1,13087,73,289892,0,3927
BPIC13,3,819-7554,11-27,4068-81524,7,162-2257
BPIC15,5,832-1409,417-491,46110-62667,6,232-438
BPIC17,2,31509-42995,17-53,285211-1269176,2,9398-13193
Gigantic,4,5000,152-157,38774-42711,1-4,1499-1553
Huge,4,5000,109,46919-53627,1-4,1416-1479
Large,4,5000,85,61789-67524,1-4,1482-1529
Medium,4,5000,65,38990-41991,1-4,1459-1550
P2p,4,5000,27,48477-53193,1-4,1430-1563
Paper,1,5000,27,66814,1,1466


In [ ]:
print(res.to_latex())

\begin{tabular}{lrlllll}
\toprule
{} &  \# Logs &      \# Cases & \# Activities &        \# Events & \# Attributes & \# Anomalies \\
\midrule
BPIC12   &       1 &        13087 &           73 &          289892 &            0 &        3927 \\
BPIC13   &       3 &     819-7554 &        11-27 &      4068-81524 &            7 &    162-2257 \\
BPIC15   &       5 &     832-1409 &      417-491 &     46110-62667 &            6 &     232-438 \\
BPIC17   &       2 &  31509-42995 &        17-53 &  285211-1269176 &            2 &  9398-13193 \\
Gigantic &       4 &         5000 &      152-157 &     38774-42711 &          1-4 &   1499-1553 \\
Huge     &       4 &         5000 &          109 &     46919-53627 &          1-4 &   1416-1479 \\
Large    &       4 &         5000 &           85 &     61789-67524 &          1-4 &   1482-1529 \\
Medium   &       4 &         5000 &           65 &     38990-41991 &          1-4 &   1459-1550 \\
P2p      &       4 &         5000 &           27 &     48477-53193

/tmp/ipykernel_4188149/138774964.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(res.to_latex())
